<a href="https://colab.research.google.com/github/LazaroR-u/Final_project_QxQ_23-24/blob/main/Final_project_QubitxQubit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
pip install cirq

In [2]:
import numpy as np
import cirq

## QUANTUM BB84 PROTOCOL

Example program to demonstrate BB84 QKD Protocol

BB84 is a quantum key distribution (QKD) protocol developed by
Charles Bennett and Gilles Brassard in 1984. It was the first quantum
cryptographic protocol, using the laws of quantum mechanics (specifically,
no-cloning) to provide provably secure key generation.

BB84 relies on the fact that it is impossible to gain information
distinguishing two non-orthogonal states without disturbing the signal.

The scheme involves two parties Alice and Bob connected by a classical
communication channel. In addition to this, Alice can also prepare
qubits in a particular state and send them to Bob using a unidirectional
quantum channel.

Alice generates two random binary strings a and b of the same length n.
The string a encodes the state and the string b encodes the basis.
She then prepares n qubits according to the following prescription:

|q[i]⟩ = |0⟩ if a[i] == 0 and b[i] == 0

|q[i]⟩ = |1⟩ if a[i] == 1 and b[i] == 0

|q[i]⟩ = |+⟩ if a[i] == 0 and b[i] == 1

|q[i]⟩ = |-⟩ if a[i] == 1 and b[i] == 1


where |+/-⟩ = 1/sqrt(2)*(|0⟩+/-|1⟩).

Alice sends her qubits to Bob. Bob then generates a random binary string
c of length n. He measures the qubit |q[i]⟩ in the {|0⟩, |1⟩} basis
(computational basis) if c[i] == 0 and in the {|+⟩,|-⟩} basis
(Hadamard basis) if c[i] == 1 and stores the result in a string m.
Alice and Bob then announce the strings b and c, which encode
the random basis choices of Alice and Bob respectively.

The strings a and m match in the places where b and c are the same.
This happens because the state was measured in the same basis in
which it was prepared. For the remaining bits, the results are
uncorrelated. The bits from strings a and m where the bases match
can be used as a key for cryptography.

BB84 is secure against intercept-and-resend attacks. The no-cloning
theorem [2] guarantees that a qubit that is in an unknown state to
begin with cannot be copied or cloned. Thus, any measurement will
destroy the initial state of the qubit. Suppose an eavesdropper Eve
intercepts all of Alice's qubits, measures them in a randomly chosen
basis, prepares another qubit in the state that she measured and resends
it to Bob. The state Eve measures is not necessarily the state Alice
prepared,  and hence, Alice and Bob will not measure the same outcome
for that qubit even if their basis choices match. Thus, Alice and Bob
can detect eavesdropping by comparing a few bits from their
obtained keys.

[1]: https://en.wikipedia.org/wiki/BB84


[2]: https://en.wikipedia.org/wiki/No-cloning_theorem



In [3]:
def main(num_qubits=12):
    # Setup non-eavesdropped protocol
    print('Simulating non-eavesdropped protocol')
    qubits = cirq.LineQubit.range(num_qubits)
    alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
    alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
    bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

    expected_key = bitstring(
        [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    circuit = make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state)

    # Run simulations.
    repetitions = 1

    result = cirq.Simulator().run(program=circuit, repetitions=repetitions)
    result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])

    # Take only qubits where bases match
    obtained_key = ''.join(
        [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    assert expected_key == obtained_key, "Keys don't match"

    print(circuit)
    print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key)

    if expected_key == obtained_key:
      print(f"No eavesdropping detected. Take the secure key: {obtained_key}")
    else:
      print("Eavesdropping detected. The key is not secure. Try again!")
      print(f"Expected key: {expected_key}, Obtained key: {obtained_key}")

    # Setup eavesdropped protocol
    print("-----------------------------------------------------------")
    print('Simulating eavesdropped protocol')
    np.random.seed(200)  # Seed random generator for consistent results
    alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
    alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
    bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
    eve_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

    expected_key = bitstring(
        [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    # Eve intercepts the qubits

    alice_eve_circuit = make_bb84_circ(num_qubits, alice_basis, eve_basis, alice_state)

    # Run simulations.
    repetitions = 1
    result = cirq.Simulator().run(program=alice_eve_circuit, repetitions=repetitions)
    eve_state = [int(np.ravel(result.measurements[str(q)])[0]) for q in qubits]

    eve_bob_circuit = make_bb84_circ(num_qubits, eve_basis, bob_basis, eve_state)

    # Run simulations.
    repetitions = 1
    result = cirq.Simulator().run(program=eve_bob_circuit, repetitions=repetitions)
    result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])


    # Take only qubits where bases match
    obtained_key = ''.join(
        [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
    )

    assert expected_key != obtained_key, "Keys shouldn't match"

    circuit = alice_eve_circuit + eve_bob_circuit

    print(circuit)
    print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key)

    if expected_key == obtained_key:
      print(f"No eavesdropping detected. Take the secure key: {obtained_key}")
    else:
      print("Eavesdropping detected. The key is not secure. Try again!")
      print(f"Expected key: {expected_key}, Obtained key: {obtained_key}")



def make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state):

    qubits = cirq.LineQubit.range(num_qubits)

    circuit = cirq.Circuit()

    # Alice prepares her qubits
    alice_enc = []
    for index, _ in enumerate(alice_basis):
        if alice_state[index] == 1:
            alice_enc.append(cirq.X(qubits[index]))
        if alice_basis[index] == 1:
            alice_enc.append(cirq.H(qubits[index]))

    circuit.append(alice_enc)

    # Bob measures the received qubits
    bob_basis_choice = []
    for index, _ in enumerate(bob_basis):
        if bob_basis[index] == 1:
            bob_basis_choice.append(cirq.H(qubits[index]))

    circuit.append(bob_basis_choice)
    circuit.append(cirq.measure_each(*qubits))

    return circuit


def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

def int_to_bin(n: int, zeros: int ):
    """
    Transform an integer to binary format.
    Parameters:
    n (int): The number of bits to generate.
    zeros (int): number of qubits.
    Returns:
    np.ndarray: An array of random bits.
    """
    return bin(n)[2:].zfill(zeros)

def print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key):
    num_qubits = len(alice_basis)
    basis_match = ''.join(
        ['X' if alice_basis[i] == bob_basis[i] else '_' for i in range(num_qubits)]
    )
    alice_basis_str = "".join(['C' if alice_basis[i] == 0 else "H" for i in range(num_qubits)])
    bob_basis_str = "".join(['C' if bob_basis[i] == 0 else "H" for i in range(num_qubits)])

    print(f'Alice\'s bases:\t{alice_basis_str}')
    print(f'Bob\'s bases:\t{bob_basis_str}')
    print(f'Alice\'s bits:\t{bitstring(alice_state)}')
    print(f'Bases match:\t{basis_match}')
    print(f'Expected key:\t{expected_key}')
    print(f'Actual key:\t{obtained_key}')
    print(f"Keys are the same:\t{expected_key == obtained_key}")


if __name__ == "__main__":
    main()

Simulating non-eavesdropped protocol
0: ────H───M───────────

1: ────H───H───M───────

2: ────X───H───H───M───

3: ────M───────────────

4: ────H───M───────────

5: ────X───H───H───M───

6: ────H───M───────────

7: ────H───M───────────

8: ────H───M───────────

9: ────H───M───────────

10: ───X───M───────────

11: ───X───H───H───M───
Alice's bases:	HHHCHHCCHHCH
Bob's bases:	CHHCCHHHCCCH
Alice's bits:	001001000011
Bases match:	_XXX_X____XX
Expected key:	010111
Actual key:	010111
Keys are the same:	True
No eavesdropping detected. Take the secure key: 010111
-----------------------------------------------------------
Simulating eavesdropped protocol
0: ────X───H───M───────X───H───M───────

1: ────X───H───M───────H───M───────────

2: ────M───────────────H───M───────────

3: ────H───M───────────X───H───M───────

4: ────M───────────────H───M───────────

5: ────X───H───H───M───X───H───M───────

6: ────X───H───M───────H───M───────────

7: ────X───H───H───M───X───H───H───M───

8: ────X───H───H─

## CREATE KEY

In [4]:
def create_key_bb84(num_qubits=100, eavesdropper=False):
    qubits = cirq.LineQubit.range(num_qubits)

    if not eavesdropper:
        alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
        alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
        bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

        expected_key = ''.join(
            [str(alice_state[i]) for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        circuit = make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state)

        result = cirq.Simulator().run(program=circuit, repetitions=1)
        result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])

        obtained_key = ''.join(
            [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        if expected_key == obtained_key:
          return expected_key
        else:
          print("Eavesdropping detected. The key is not secure. Try again!")
          print(f"Expected key: {expected_key}, Obtained key: {obtained_key}")

    elif eavesdropper:
        alice_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
        alice_state = [np.random.randint(0, 2) for _ in range(num_qubits)]
        bob_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]
        eve_basis = [np.random.randint(0, 2) for _ in range(num_qubits)]

        expected_key = bitstring(
            [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        # Eve intercepts the qubits
        alice_eve_circuit = make_bb84_circ(num_qubits, alice_basis, eve_basis, alice_state)

        # Run simulations.
        repetitions = 1
        result = cirq.Simulator().run(program=alice_eve_circuit, repetitions=repetitions)
        eve_state = [int(np.ravel(result.measurements[str(q)])[0]) for q in qubits]

        eve_bob_circuit = make_bb84_circ(num_qubits, eve_basis, bob_basis, eve_state)

        # Run simulations.
        repetitions = 1
        result = cirq.Simulator().run(program=eve_bob_circuit, repetitions=repetitions)
        result_bitstring = bitstring([int(np.ravel(result.measurements[str(q)])[0]) for q in qubits])

        # Take only qubits where bases match
        obtained_key = ''.join(
            [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]]
        )

        if expected_key == obtained_key:
          print(f"No eavesdropping detected. Take the secure key: {obtained_key}")
        else:
          print("Eavesdropping detected. The key is not secure. Try again!")
          print(f"Expected key: {expected_key}, Obtained key: {obtained_key}")

create_key_bb84(eavesdropper=False)

'010101000000000111101010000100001101111111'

In [5]:
create_key_bb84(eavesdropper=True)

Eavesdropping detected. The key is not secure. Try again!
Expected key: 1010000101011010110110011011001001110010011000111111, Obtained key: 1010101011110010110110011001001111010011111000000110


## ENCRYPT - DECRYPT - TRANSFORM FUNCTIONS

In [12]:
def shorten_key(data, key):
    # The key should be the same length as the data for the binary addition
    if len(key) < len(data):
        repeat_factor = (len(data) // len(key)) + 1
        new_key = (key * repeat_factor)[:len(data)]
        return new_key
    else:
        return key


def encryption(data,key):
    # function which creates the encrypted message using the key created between Alice and Bob
    if len(data) != len(key):
        # if the key is longer than the data, shorten the key
        key = shorten_key(data,key)

    encrypted_message = [None for j in range(len(data))]

    # binary addition
    for i in range(len(data)):
      if (data[i] == str(0) and key[i] == str(0)) or (data[i] == str(1) and key[i] == str(1)):
        encrypted_message[i] = 0
      elif (data[i] == str(1) and key[i] == str(0)) or (data[i] == str(0) and key[i] == str(1)):
        encrypted_message[i] = 1

    return ''.join(map(str, encrypted_message))


def decryption(message,key):
    # function which recreates the original message using the key created between Alice and Bob
    # The message argument here is the encrypted message Bob recieves from Alice
    # This function is the same as the encryption function, using simple binary addition
    # Used by Bob
    return encryption(message,key)

#Transform a list into a string
def bin_to_num(lista):
  return int(''.join(map(str, lista)), 2)

In [13]:
def binary_converter(string):
    # Initialize an empty list to store the binary values
    binary_list = []

    # Iterate through each character in the string
    for char in string:
        # Convert the character to its ASCII integer value and then to its binary representation,
        # and append it to the list
        binary_list.append(format(ord(char), '08b'))

    # Join the binary values in the list and return as a single string
    return ''.join(binary_list)


def string_converter(binary):
    # Initialize an empty string to store the result
    result = ""

    # Iterate through the binary string in steps of 8 characters
    for i in range(0, len(binary), 8):
        # Extract the 8-character binary block
        binary_block = binary[i:i + 8]

        # Convert the binary block to its decimal value and then to its corresponding ASCII character
        char = chr(int(binary_block, 2))

        # Add the character to the result
        result += char

    return result

# EXAMPLES

## Number: like a NIP

In [16]:
password = 12345

message = int_to_bin(password, 14)

num_qubits = 2*len(message)
key = create_key_bb84(num_qubits)
print(f"key: {key}, len: {len(key)}")
key = shorten_key(message, key)


encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = bin_to_num(decrypted_message)

print(f"sent message: {password}, messagge in binary format: {message}")
print(f"key: {key}, len: {len(key)}; message: {message}, len: {len(message)}")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print(f"received message: {received_message}")

key: 110010001000, len: 12
sent message: 12345, messagge in binary format: 11000000111001
key: 11001000100011, len: 14; message: 11000000111001, len: 14
encrypted message: 00001000011010
decrypted message: 11000000111001
received message: 12345


## Word: like a PASSWORD




In [9]:
#text
original_message = "Google"

message = binary_converter(original_message)
message = ''.join(map(str, message))

num_qubits = len(message)
key = create_key_bb84()
key = shorten_key(message, key)

encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = string_converter(decrypted_message)

print(f"sent message: {original_message}, messagge in binary format: {message}")
print(f"key: {key}, len: {len(key)}; message: {message}, len: {len(message)}")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print(f"received message: {received_message}")

sent message: Google, messagge in binary format: 010001110110111101101111011001110110110001100101
key: 010000001001010000001011010110101100011100101010, len: 48; message: 010001110110111101101111011001110110110001100101, len: 48
encrypted message: 000001111111101101100100001111011010101101001111
decrypted message: 010001110110111101101111011001110110110001100101
received message: Google


In [10]:
#text
original_message = "HswrTY@K"

message = binary_converter(original_message)
message = ''.join(map(str, message))

num_qubits = len(message)
key = create_key_bb84(num_qubits)
key = shorten_key(message, key)


encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = string_converter(decrypted_message)

print(f"sent message: {original_message}, messagge in binary format: {message}")
print(f"key: {key}, len: {len(key)}; message: {message}, len: {len(message)}")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print(f"received message: {received_message}")

sent message: HswrTY@K, messagge in binary format: 0100100001110011011101110111001001010100010110010100000001001011
key: 1101001101110001000110100110100001111010011011100010001101001101, len: 64; message: 0100100001110011011101110111001001010100010110010100000001001011, len: 64
encrypted message: 1001101100000010011011010001101000101110001101110110001100000110
decrypted message: 0100100001110011011101110111001001010100010110010100000001001011
received message: HswrTY@K


## Message: like a Whatsapp message

In [11]:
original_message = "I am a future quantum leader (:"
print(f"original message: {original_message}")


message = binary_converter(original_message)
message = ''.join(map(str, message))

print("---------------------------------------------")

print(f"binary message: {message}, len: {len(message)}")

num_qubits = 2*len(message)
key = create_key_bb84(num_qubits)

print(f"original key: {key}, len: {len(key)}")

key = shorten_key(message, key)

print(f"adjusted key: {key}, len: {len(key)}")



encrypted_message = encryption(message, key)
decrypted_message = decryption(encrypted_message, key)
received_message = string_converter(decrypted_message)


print("---------------------------------------------")
print(f"encrypted message: {encrypted_message}")
print(f"decrypted message: {decrypted_message}")
print("---------------------------------------------")
print(f"received message: {received_message}")


original message: I am a future quantum leader (:
---------------------------------------------
binary message: 01001001001000000110000101101101001000000110000100100000011001100111010101110100011101010111001001100101001000000111000101110101011000010110111001110100011101010110110100100000011011000110010101100001011001000110010101110010001000000010100000111010, len: 248
original key: 0101010111111100110100111000011100000001001000001100111110110001101100011100110111101000001110100101011000110101111000000011100101000111011011101101001111011001111100000101110011110111111101100101011010100111110100011000101000001101001100010001000101110101000100, len: 262
adjusted key: 0101010111111100110100111000011100000001001000001100111110110001101100011100110111101000001110100101011000110101111000000011100101000111011011101101001111011001111100000101110011110111111101100101011010100111110100011000101000001101001100010001000101110101000100, len: 262
---------------------------------------------
encrypted